# Caso Práctico - Urbvan Travel BIA

La información es ficticia, y representaría datos de reservaciones durante el mes de abril de este año.

En función de la data proporcionada contesta la siguiente información:
* Identifica los tres problemas más relevantes que observaste y por qué (justificar con data)
* Menciona tres iniciativas que implementarías para solucionar el problema (desarrolla la estrategia para que se comunique con el equipo) y por qué (producto, growth, operativo, etc)
* Qué información adicional te gustaría conocer de los usuarios para llevar a cabo mejores iniciativas
* Si pudieras tener acceso a una respuesta en específico (con todo la info que lo respalde), cuál sería?

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

original_df = pd.read_csv('data.csv')
df = original_df.copy()

In [2]:
df.dropna(subset=['reservation_id'], inplace=True) # dropping rows without 'reservation_id'

web = df[df.source == 'Web'].index
df.drop(web, inplace=True) # dropping rows with 'source' == 'Web' (only 9 rows)

unconfirmed = df[df.reservation_confirmed == False].index
df.drop(unconfirmed, inplace=True) # dropping rows with 'reservation_confirmed' == False (296 rows)

In [3]:
df.drop([
    'source', # all (except 9) are 'App'
    'reservation_confirmed', # all are (except 296) are True
    'reservation_finished', # all are False
    'successfully_charged', # all are True
    'reservation_active', # all are True
    'trip_month', # all are 2021-04-01
    'trip_period', # all are 2021-04-01
], axis=1, inplace=True) # droppping columns with only one value

In [5]:
date_cols = [
    'reservation_created_at',
    'trip_week',
    'trip_day',
    'trip_itinerary',
    'client_first_reservation',
    'client_prev_reservation',
    'client_next_reservation',
    'client_last_reservation',
    'client_first_trip',
    'client_prev_trip',
    'client_next_trip',
    'client_last_trip'
]


for col in date_cols:
    df[col] = df[col].apply(lambda x: pd.to_datetime(x, dayfirst=True))
    
df.trip_itinerary = df.trip_itinerary.dt.time

In [6]:
new_types = {
    'reservation_id':'int64',
    'client_id':'int64',
    'res_trip_id':'int64',
    'stop_id':'int64',
    'pickup_stop_id':'int64',
    'trip_id':'int64',
    'van_id':'int64'
}

df = df.astype(dtype=new_types)

In [7]:
df.reset_index(drop=True, inplace=True)
df

,reservation_id,cost,seats,reservation_created_at,client_id,res_trip_id,stop_id,pickup_stop_id,trip_id,trip_week,...,client_first_trip,client_prev_trip,client_next_trip,client_last_trip,days_between_trips,res_to_trip_days,client_first_route,client_prev_route,client_next_route,client_last_route
0,2672592,214.0,2.0,2021-03-31 10:42:00,1027,492245,16522,16516,492245,2021-03-29,...,2021-04-02 11:00:00,NaT,2021-04-02 19:45:00,2021-04-02 19:45:00,NaN,2.012405,CDMX - Tepoztlán - Cuernavaca,NaN,Cuernavaca - Tepoztlán - CDMX,CDMX - Tepoztlán - Cuernavaca
1,2673360,107.0,1.0,2021-04-01 16:50:00,1266,492245,16519,16516,492245,2021-03-29,...,2021-04-02 11:00:00,NaT,2021-04-03 16:00:00,2021-04-03 16:00:00,NaN,0.756848,CDMX - Tepoztlán - Cuernavaca,NaN,Cuernavaca - Tepoztlán - CDMX,CDMX - Tepoztlán - Cuernavaca
2,2673997,214.0,2.0,2021-04-03 11:16:00,1266,493244,16515,16512,493244,2021-03-29,...,2021-04-02 11:00:00,2021-04-02 11:00:00,NaT,2021-04-03 16:00:00,1.208333,0.196736,CDMX - Tepoztlán - Cuernavaca,CDMX - Tepoztlán - Cuernavaca,NaN,Cuernavaca - Tepoztlán - CDMX
3,2674442,180.0,1.0,2021-04-04 15:33:00,1538,496586,16106,16101,496586,2021-04-05,...,2021-01-18 06:20:00,2021-03-31 19:35:00,2021-04-10 05:00:00,2021-05-21 15:00:00,4.447917,0.615746,Querétaro - CDMX,CDMX - Querétaro,CDMX - Querétaro,Querétaro - CDMX
4,2678137,195.0,1.0,2021-04-09 19:57:00,1538,498540,16100,16095,498540,2021-04-05,...,2021-01-18 06:20:00,2021-04-05 06:20:00,2021-04-12 06:20:00,2021-05-21 15:00:00,4.944444,0.376744,Querétaro - CDMX,Querétaro - CDMX,Querétaro - CDMX,CDMX - Querétaro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3232,2691649,190.0,1.0,2021-04-30 13:53:00,418487,513869,17753,17749,513869,2021-04-26,...,2021-04-30 17:30:00,NaT,NaT,2021-04-30 17:30:00,NaN,0.150243,Querétaro - CDMX,NaN,NaN,Querétaro - CDMX
3233,2691683,260.0,2.0,2021-04-30 14:43:00,418497,513911,18047,18044,513911,2021-04-26,...,2021-04-30 16:00:00,NaT,2021-05-02 19:30:00,2021-05-02 19:30:00,NaN,0.053221,Cuernavaca - Tepoztlán - CDMX,NaN,CDMX - Tepoztlán - Cuernavaca,Cuernavaca - Tepoztlán - CDMX
3234,2691687,260.0,2.0,2021-04-30 15:02:00,418505,513911,18047,18044,513911,2021-04-26,...,2021-04-30 16:00:00,NaT,2021-05-02 19:30:00,2021-05-02 19:30:00,NaN,0.039622,Cuernavaca - Tepoztlán - CDMX,NaN,CDMX - Tepoztlán - Cuernavaca,Cuernavaca - Tepoztlán - CDMX
3235,2691808,185.0,1.0,2021-04-30 18:01:00,418562,517529,17745,17741,517529,2021-04-26,...,2021-04-30 18:36:00,NaT,NaT,2021-04-30 18:36:00,NaN,0.024039,CDMX - Querétaro,NaN,NaN,CDMX - Querétaro


In [8]:
df.dtypes

reservation_id                       int64
cost                               float64
seats                              float64
reservation_created_at      datetime64[ns]
client_id                            int64
res_trip_id                          int64
stop_id                              int64
pickup_stop_id                       int64
trip_id                              int64
trip_week                   datetime64[ns]
trip_day                    datetime64[ns]
trip_itinerary                      object
trip_seat_inventory                  int64
van_id                               int64
seat_price                           int64
route_name                          object
route_active                          bool
km_google                          float64
line_name                           object
trip_cpt                           float64
client_first_reservation    datetime64[ns]
client_prev_reservation     datetime64[ns]
client_next_reservation     datetime64[ns]
client_last